In [1]:
from itertools import permutations
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

/home/kd/anaconda3/envs/csci544/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [122]:
def get_candidates_setwise(Ci,Ak,Sk):
    Ci_final = set()
    for cj in Ci:
        # Ci.remove(cj)
        for a in Ak:
            c0j = cj.split()
            for i, word in zip(Sk, a):
                if type(word) == int:
                    print(Ak,Sk)
                    print('i',i)
                    print('word',word)
                c0j[i] = (word)
                # print(c0j)
            Ci_final.add(' '.join((c0j)))
    return Ci_final

In [123]:
from itertools import permutations

def candidate_set_generation(xi, pi0, Ti, T0i, F):
    # Initialize candidate sentence with masked tokens
    c0 = ' '.join(['<mask>'] * len(pi0))
    # print(f'pi0 {pi0}')
    # print(f' c0 {c0}' )
    Ci = {c0}
    # Ci = [c0]
    # print(f'Ci {Ci}')
    Tshared = Ti.intersection(T0i)
    # print(f'T shared {Tshared}')

    for tk in Tshared:
        print(tk)

        Wk = [word for word, pos in xi if pos == tk]
        # print(f'Wk {Wk}')
    
        Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
        # print(f'Sk {Sk}')
        # print(f'len(Wk) {len(Wk)} len(Sk) {len(Sk)}')
        if len(Wk)==1:
            Ak = [Wk]
        # elif len(Sk)==1: 
        #     Ak = [(i) for i in Wk]
        elif len(Wk)<=len(Sk):
            Ak = list(permutations(Sk,len(Wk)))
            # print(f'Ak {Ak}')
        else:
            Ak = list(permutations(Wk,len(Sk)))
            # print(f'Ak {Ak}')
        if not Ak:
            continue

        Ci_tag = get_candidates_setwise(Ci,Ak,Sk)
        Ci = Ci.union(Ci_tag)
    return Ci


In [134]:
# non-offensive sentence
pi0 = 'it is not they happen all the time there are thousands of train derailments a year but when a particularly bad one happens news outlets jump on the trend and start reporting more they are not more frequent this is just the first a lot of people are hearing about them edit you all can downvote but i am right look it up if you are really concerned you should know this'
pi0 = pi0.split(' ')[:10]
# non-offensive sentence pos tags 
tag_pi0 = ['PRP', 'VBZ', 'RB', 'PRP', 'VBP', 'PDT', 'DT', 'NN', 'EX', 'VBP', 'NNS', 'IN', 'NN', 'NNS', 'DT', 'NN', 'CC', 'WRB', 'DT', 'RB', 'JJ', 'NN', 'VBZ', 'NN', 'NNS', 'VB', 'IN', 'DT', 'NN', 'CC', 'VBP', 'VBG', 'JJR', 'PRP', 'VBP', 'RB', 'RBR', 'JJ', 'DT', 'VBZ', 'RB', 'DT', 'JJ', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'VBG', 'IN', 'PRP', 'NN', 'PRP', 'DT', 'MD', 'VB', 'CC', 'PRP', 'VBP', 'JJ', 'VB', 'PRP', 'RP', 'IN', 'PRP', 'VBP', 'RB', 'JJ', 'PRP', 'MD', 'VB', 'DT']
tag_pi0 = tag_pi0[:10]

# offensive sentence 
xi = 'i get this feeling that derailments happen a lot and we are only paying attention now because of what happened in ohio that said why the fuck do they happen so often'
# offensive sentence pos sequence 
tag_xi = ['PRP', 'VBP', 'DT', 'NN', 'IN', 'NNS', 'VBP', 'DT', 'NN', 'CC', 'PRP', 'VBP', 'RB', 'VBG', 'NN', 'RB', 'IN', 'IN', 'WP', 'VBD', 'IN', 'NNP', 'WDT', 'VBD', 'WRB', 'DT', 'BW', 'VBP', 'PRP', 'VB', 'RB', 'RB']


# unique pos tags/tokens of non-offensive sentence
T0i = set(tag_pi0)
# unique pos tags/tokens of offensive sentence
Ti = set(tag_xi)


# tuple set of word in offensive sentence and its corresponding pos tag
xi_mod = [(word,tag) for word,tag in zip(xi.split(),tag_xi)]
# tuple set of word in non-offensive sentence and its corresponding pos tag
pi0_mod = [(word,tag) for word,tag in zip(pi0,tag_pi0)]
candidate_sentences = candidate_set_generation(xi=xi_mod,pi0=pi0_mod,Ti=Ti,T0i=T0i,F=None)
print(len(candidate_sentences))


VBP
RB
DT
NN
PRP
7280


In [120]:
model = RobertaForMaskedLM.from_pretrained('distilroberta-base')
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')

# sequence = f"What the {tokenizer.mask_token} is going on?" # "The world will end in <mask>"
# sequence = f"they <mask> so i happen <mask> this attention <mask> are <mask> that feeling <mask> a"
def fill_mask_topk(sequence, k=5):
    input_seq = tokenizer.encode(sequence, return_tensors='pt') # tensor([[0, 133, 232, 40, 253, 11, 50264, 2]])
    mask_token_index = torch.where(input_seq == tokenizer.mask_token_id)[1] # (tensor([0]), tensor([6])) - we only want the the 2nd dimension
    token_logits = model(input_seq).logits
    masked_token_logits = token_logits[0, mask_token_index, :]

    top_k_tokens = torch.topk(masked_token_logits, k, dim=1).indices[0].tolist()
    for token in top_k_tokens:
        print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

In [139]:
for i,sentence in enumerate(list(candidate_sentences)[:50]):
    print(f'{i+1}. {sentence}')
    fill_mask_topk((sentence),k=2)

1. <mask> <mask> only <mask> get <mask> <mask> <mask> <mask> do
You You only You get You You You You do
Your Your only Your get Your Your Your Your do
2. i <mask> <mask> we do <mask> the attention <mask> happen
i Pad Pad we do Pad the attention Pad happen
i Stock Stock we do Stock the attention Stock happen
3. they <mask> now i happen <mask> a feeling <mask> get
they  know now i happen  know a feeling  know get
they  dont now i happen  dont a feeling  dont get
4. they <mask> now we happen <mask> this <mask> <mask> get
they  know now we happen  know this  know  know get
they  say now we happen  say this  say  say get
5. i <mask> <mask> they get <mask> <mask> lot <mask> do
i  dont  dont they get  dont  dont lot  dont do
i Pad Pad they get Pad Pad lot Pad do
6. they <mask> now we happen <mask> the <mask> <mask> are
they  know now we happen  know the  know  know are
they  say now we happen  say the  say  say are
7. they <mask> often i get <mask> a <mask> <mask> are
they  dont often i get  